#

# 12. Macine Learning Model Deployment

## CPE 490 590
## Rahul Bhadani

# 1. Creating a PyTorch Model for IRIS Classification

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import onnx
import onnxruntime as ort

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# 2. PyTorch Model - Model Definition and Training

In [8]:
class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.layer1 = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10, 3)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x
        
model = IrisClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 55
for epoch in range(epochs):
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/55], Loss: 0.8916
Epoch [20/55], Loss: 0.6242
Epoch [30/55], Loss: 0.4371
Epoch [40/55], Loss: 0.3273
Epoch [50/55], Loss: 0.2546


# 3. PyTorch Model - Evaluation and ONNX Export

In [12]:
# Evaluate the model
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f'Test Accuracy: {accuracy:.4f}')

# Export the model to ONNX
dummy_input = torch.randn(1, 4)
input_names = ["input"]
output_names = ["output"]

torch.onnx.export(
    model, dummy_input, "iris_classifier.onnx",
    input_names=input_names, output_names=output_names,
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11  # Try a lower opset version like 11
)
print("Model saved as iris_classifier.onnx")

Test Accuracy: 1.0000
Model saved as iris_classifier.onnx


# 4. PyTorch Model - Saving Scaler Parameters and Testing ONNX

In [13]:
# Save scaler mean and scale for later use in the production application
import json
with open("scaler_params.json", "w") as f:
    json.dump({ "mean": scaler.mean_.tolist(), "scale": scaler.scale_.tolist()
    }, f)

print("Scaler parameters saved as scaler_params.json")
# Test the ONNX model
ort_session = ort.InferenceSession("iris_classifier.onnx")
# Prepare input
ort_inputs = {ort_session.get_inputs()[0].name: X_test.astype(np.float32)}
# Run inference
ort_outputs = ort_session.run(None, ort_inputs)
# Calculate accuracy
ort_predicted = np.argmax(ort_outputs[0], axis=1)
ort_accuracy = np.sum(ort_predicted == y_test) / len(y_test)
print(f'ONNX Model Test Accuracy: {ort_accuracy:.4f}')

Scaler parameters saved as scaler_params.json
ONNX Model Test Accuracy: 1.0000


# 5. PyTorch Model - Saving Example Data

In [16]:
# Save some example data for testing in Production (unscaled Data)
examples = []
class_names = iris.target_names
for i in range(10):
    idx = np.random.randint(0, len(X))
    examples.append({
        "features": X[idx].tolist(),
        "label": int(y[idx]),
        "class_name": class_names[y[idx]]
    })

with open("example_data.json", "w") as f:
    json.dump(examples, f)

print("Example data saved as example_data.json")

Example data saved as example_data.json
